In [1]:
from tools import tools
import pandas as pd

In [8]:
from sentence_transformers import SentenceTransformer, util
from torch import device, cuda, save, load
device = device('cuda' if cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
reports_file_path = 'data/br/firefox/firefox.csv'
relations_file_path = 'data/br/firefox/firefox_pairs.csv'

reports_firefox = pd.read_csv(reports_file_path, index_col='bug_id')
relations_firefox = pd.read_csv(relations_file_path)

In [4]:
test_set = reports_firefox[0:1000]
test_set.head()

,priority,component,dup_id,short_desc,description,bug_status,resolution,version,creation_ts,delta_ts
bug_id,,,,,,,,,,
10954,P3,Preferences,[],Dialup properties needs to be exposed in prefs,The dialup properties of the profile should be...,RESOLVED,WONTFIX,Trunk,1999-07-30 15:55:51 -0700,2008-05-14 11:44:15 -0700
14871,--,General,269442,[Find] Find whole word only,Please add Match Whole Word Only option to bro...,RESOLVED,DUPLICATE,Trunk,1999-09-24 14:49:34 -0700,2011-10-05 16:35:31 -0700
19118,--,Preferences,[],Plug-In Manager (ui for choosing mimetype-plug...,I would really like a plug-in manager for my b...,RESOLVED,WONTFIX,Trunk,1999-11-17 14:58:26 -0800,2013-01-29 11:48:39 -0800
54746,P3,Preferences,[],Language encodings in font prefs dialog not so...,Language encodings are listed in a seemingly r...,RESOLVED,WORKSFORME,Trunk,2000-09-29 14:12:11 -0700,2013-02-27 15:47:29 -0800
56892,P3,General,[],Synaptics touchpad scrolling not working,From Bugzilla Helper:; User-Agent: Mozilla/5.0...,RESOLVED,WORKSFORME,unspecified,2000-10-16 14:48:15 -0700,2009-10-14 11:38:29 -0700


In [5]:
model_name = 'all-mpnet-base-v2' 

In [6]:
test_set = tools.generate_encodings(test_set, model_name)
test_set[model_name].to_csv('data/subset_embeddings.csv')

c:\Users\lpros\Documents\thesis\thesis-code\fine-tuning-experimentation\tools.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[model_name] = df['description'].map(lambda x: tools.encode_description(model, x))


In [7]:
tensor_dict = test_set[model_name].to_dict()
save(tensor_dict, 'data/subset_torch_file.pt')

In [9]:
tensor_dict = load('data/subset_torch_file.pt')

In [11]:
test_set.head()

,priority,component,dup_id,short_desc,description,bug_status,resolution,version,creation_ts,delta_ts,all-mpnet-base-v2
bug_id,,,,,,,,,,,
10954,P3,Preferences,[],Dialup properties needs to be exposed in prefs,The dialup properties of the profile should be...,RESOLVED,WONTFIX,Trunk,1999-07-30 15:55:51 -0700,2008-05-14 11:44:15 -0700,"[-0.011274192, 0.09213826, -0.010780337, -0.03..."
14871,--,General,269442,[Find] Find whole word only,Please add Match Whole Word Only option to bro...,RESOLVED,DUPLICATE,Trunk,1999-09-24 14:49:34 -0700,2011-10-05 16:35:31 -0700,"[0.02882544, 0.049673, -0.0020014888, 0.043006..."
19118,--,Preferences,[],Plug-In Manager (ui for choosing mimetype-plug...,I would really like a plug-in manager for my b...,RESOLVED,WONTFIX,Trunk,1999-11-17 14:58:26 -0800,2013-01-29 11:48:39 -0800,"[0.0067006177, -0.0040191724, -0.031814296, 0...."
54746,P3,Preferences,[],Language encodings in font prefs dialog not so...,Language encodings are listed in a seemingly r...,RESOLVED,WORKSFORME,Trunk,2000-09-29 14:12:11 -0700,2013-02-27 15:47:29 -0800,"[0.013975114, -0.027381923, 0.012256341, 0.063..."
56892,P3,General,[],Synaptics touchpad scrolling not working,From Bugzilla Helper:; User-Agent: Mozilla/5.0...,RESOLVED,WORKSFORME,unspecified,2000-10-16 14:48:15 -0700,2009-10-14 11:38:29 -0700,"[0.002355588, 0.039769035, 0.0009729369, 0.016..."


In [26]:
def generate_encodings(dataset: pd.DataFrame, model_name: str):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(
        dataset['description'].tolist(),
        convert_to_tensor=True
    )

    embeddings_map = {
        bug_id: tensor for bug_id, tensor in zip(dataset.index, embeddings)
    }

    return embeddings_map

In [27]:
emb = generate_encodings(test_set, model_name)

In [28]:
save(emb, 'data/test_tensor_map_firefox_1000.pt')

In [29]:
load_emb = load('data/test_tensor_map_firefox_1000.pt')

In [ ]:
whole_dataset_emb = generate_encodings(reports_firefox, model_name)

In [ ]:
save(whole_dataset_emb, 'data/test_tensor_map_firefox_whole_set.pt')